In [1]:
# -*- coding: utf-8 -*-
import numpy as np
import pandas as pd
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import tensorflow as tf
from tensorflow.keras import layers, models

# -------- إعدادات --------
WIDE_PATH   = "Equites_Close_Daily.csv"   # ملف wide (كل عمود = يوم)
WINDOW_LEN  = 20
TRAIN_RATIO = 0.8
EPOCHS      = 12
BATCH_SIZE  = 128

# -------- دوال --------
def to_float(x):
    try:
        return float(x)
    except:
        return np.nan

def build_sequences(series_vals, window):
    X, y = [], []
    for t in range(window, len(series_vals)):
        feat = series_vals[t-window:t]
        target = series_vals[t]
        if np.isnan(feat).any() or np.isnan(target):
            continue
        X.append(feat)
        y.append(target)
    if not X:
        return np.empty((0, window, 1)), np.empty((0,))
    X = np.array(X, dtype="float32")[:, :, None]
    y = np.array(y, dtype="float32")
    return X, y

# -------- تحميل الملف --------
wide = pd.read_csv(WIDE_PATH, encoding="utf-8-sig").set_index("اسم الشركة")
wide = wide.applymap(to_float)

# -------- إعداد بيانات التدريب --------
X_tr_list, y_tr_list = [], []
X_te_list, y_te_list = [], []
te_mu_list, te_sigma_list = [], []
next_day_preds = []

for comp, row in wide.iterrows():
    vals = row.values.astype("float64")
    if np.sum(~np.isnan(vals)) < WINDOW_LEN + 2:
        continue

    X_all, y_all = build_sequences(vals, WINDOW_LEN)
    if len(y_all) < 3:
        continue

    split = max(1, int(len(y_all) * TRAIN_RATIO))
    X_tr_c, y_tr_c = X_all[:split], y_all[:split]
    X_te_c, y_te_c = X_all[split:], y_all[split:]
    if len(y_te_c) == 0:
        continue

    # تطبيع based على train
    train_series_vals = np.concatenate([X_tr_c.reshape(-1), y_tr_c.reshape(-1)])
    mu = np.nanmean(train_series_vals)
    sigma = np.nanstd(train_series_vals)
    if not np.isfinite(mu) or sigma <= 1e-12:
        continue

    X_tr_c_norm = (X_tr_c - mu) / sigma
    y_tr_c_norm = (y_tr_c - mu) / sigma
    X_te_c_norm = (X_te_c - mu) / sigma
    y_te_c_norm = (y_te_c - mu) / sigma

    X_tr_list.append(X_tr_c_norm)
    y_tr_list.append(y_tr_c_norm)
    X_te_list.append(X_te_c_norm)
    y_te_list.append(y_te_c_norm)

    te_mu_list += [mu] * len(y_te_c_norm)
    te_sigma_list += [sigma] * len(y_te_c_norm)

    # آخر نافذة للتنبؤ
    last_window_raw = vals[-WINDOW_LEN:]
    if np.isnan(last_window_raw).any():
        next_day_preds.append({"اسم الشركة": comp, "next_day_pred": np.nan})
    else:
        last_window_norm = ((last_window_raw - mu) / sigma).astype("float32").reshape(1, WINDOW_LEN, 1)
        next_day_preds.append({
            "اسم الشركة": comp,
            "last_window_norm": last_window_norm,
            "mu": mu,
            "sigma": sigma,
            "yesterday": last_window_raw[-1]
        })

# دمج
X_train = np.concatenate(X_tr_list, axis=0)
y_train = np.concatenate(y_tr_list, axis=0)
X_test  = np.concatenate(X_te_list, axis=0)
y_test  = np.concatenate(y_te_list, axis=0)
te_mu   = np.array(te_mu_list, dtype="float64")
te_sig  = np.array(te_sigma_list, dtype="float64")

print("Train:", X_train.shape, y_train.shape)
print("Test :", X_test.shape,  y_test.shape)

# -------- RNN --------
tf.keras.backend.clear_session()
model = models.Sequential([
    layers.Input(shape=(WINDOW_LEN, 1)),
    layers.LSTM(64, return_sequences=True),
    layers.Dropout(0.2),
    layers.LSTM(32),
    layers.Dense(1)
])

model.compile(optimizer="adam", loss="mse")
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    verbose=1,
    shuffle=True
)

# -------- تقييم --------
y_hat_norm = model.predict(X_test, verbose=0).reshape(-1)
y_hat = y_hat_norm * te_sig + te_mu
y_true = y_test * te_sig + te_mu

R2   = r2_score(y_true, y_hat)
MAE  = mean_absolute_error(y_true, y_hat)
RMSE = float(np.sqrt(mean_squared_error(y_true, y_hat)))

print("\n=== Global Test Metrics ===")
print(f"R²   = {R2:.4f}")
print(f"MAE  = {MAE:.4f}")
print(f"RMSE = {RMSE:.4f}")

# -------- توقع + توصية --------
pred_rows = []
for rec in next_day_preds:
    comp = rec["اسم الشركة"]
    if "last_window_norm" not in rec:
        pred_rows.append({"اسم الشركة": comp, "yesterday": np.nan, "predicted": np.nan, "recommendation": "N/A"})
        continue
    pred_norm = model.predict(rec["last_window_norm"], verbose=0).reshape(-1)[0]
    pred = pred_norm * rec["sigma"] + rec["mu"]
    yesterday = rec["yesterday"]
    recommendation = "Buy" if pred < yesterday else "Don’t Buy"
    pred_rows.append({
        "اسم الشركة": comp,
        "yesterday": float(yesterday),
        "predicted": float(pred),
        "recommendation": recommendation
    })

pred_df = pd.DataFrame(pred_rows)
print("\nSample predictions with recommendations:")
print(pred_df.head(10))

import os

# -------- مجلد الحفظ --------
OUT_DIR = "out"
os.makedirs(OUT_DIR, exist_ok=True)   # ينشئ مجلد out إذا لم يكن موجودًا

# -------- حفظ الموديل --------
model_path = os.path.join(OUT_DIR, "rnn_model_with_norm.h5")
model.save(model_path)
print(f"✅ Saved model: {model_path}")

# -------- حفظ النتائج --------
csv_path = os.path.join(OUT_DIR, "rnn_recommendations.csv")
pred_df.to_csv(csv_path, index=False, encoding="utf-8-sig")
print(f"✅ Saved recommendations: {csv_path}")

C:\Users\asma5\AppData\Local\Temp\ipykernel_17068\4061115701.py:39: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  wide = wide.applymap(to_float)


Train: (98200, 20, 1) (98200,)
Test : (24600, 20, 1) (24600,)

Epoch 1/12
768/768 ━━━━━━━━━━━━━━━━━━━━ 18s 19ms/step - loss: 0.0302 - val_loss: 0.1460
Epoch 2/12
768/768 ━━━━━━━━━━━━━━━━━━━━ 14s 19ms/step - loss: 0.0108 - val_loss: 0.1173
Epoch 3/12
768/768 ━━━━━━━━━━━━━━━━━━━━ 14s 18ms/step - loss: 0.0094 - val_loss: 0.0892
Epoch 4/12
768/768 ━━━━━━━━━━━━━━━━━━━━ 15s 19ms/step - loss: 0.0092 - val_loss: 0.0887
Epoch 5/12
768/768 ━━━━━━━━━━━━━━━━━━━━ 15s 19ms/step - loss: 0.0090 - val_loss: 0.0781
Epoch 6/12
768/768 ━━━━━━━━━━━━━━━━━━━━ 15s 19ms/step - loss: 0.0089 - val_loss: 0.0674
Epoch 7/12
768/768 ━━━━━━━━━━━━━━━━━━━━ 15s 19ms/step - loss: 0.0089 - val_loss: 0.0897
Epoch 8/12
768/768 ━━━━━━━━━━━━━━━━━━━━ 15s 19ms/step - loss: 0.0088 - val_loss: 0.0618
Epoch 9/12
768/768 ━━━━━━━━━━━━━━━━━━━━ 15s 19ms/step - loss: 0.0087 - val_loss: 0.0763
Epoch 10/12
768/768 ━━━━━━━━━━━━━━━━━━━━ 15s 19ms/step - loss: 0.0086 - val_loss: 0.0837
Epoch 11/12
768/768 ━━━━━━━━━━━━━━━━━━━━ 15s 19ms/step -


Sample predictions with recommendations:
                                اسم الشركة  yesterday   predicted  \
0                 اعمار المدينة الاقتصادية      17.23   17.257523   
1                     البنك الاهلي السعودي      38.65   38.840885   
2                      البنك السعودي الاول      37.90   37.912212   
3                    البنك السعودي الفرنسي      19.30   19.340147   
4                  البنك السعودي للاستثمار      12.76   12.740829   
5                      البنك العربي الوطني      18.97   18.995378   
6                 الشركة التعاونية للتامين     130.20  106.493378   
7  الشركة الخليجية العامة للتامين التعاوني      13.17   13.307072   
8           الشركة السعودية لاعادة التامين      18.60   18.536388   
9            الشركة السعودية لانابيب الصلب      36.30   35.888721   

  recommendation  
0      Don’t Buy  
1      Don’t Buy  
2      Don’t Buy  
3      Don’t Buy  
4            Buy  
5      Don’t Buy  
6            Buy  
7      Don’t Buy  
8            Buy  
9       

In [2]:
# ===================== Cell 1 — Init + Load RNN =====================
import os, re, math
import pandas as pd, numpy as np, datetime as dt
from pathlib import Path

import firebase_admin
from firebase_admin import credentials, firestore
import tensorflow as tf
from tensorflow.keras.models import load_model

# --- CONFIG ---
SERVICE_ACCOUNT_JSON = r"nomu-47a92-firebase-adminsdk-fbsvc-1b2e28026c.json"   # <-- change path if needed
MODEL_PATH           = Path("out/rnn_model_with_norm.h5")   # <-- this is a Keras H5 model
COMPANIES_COLLECTION = "companies"
PRICE_SUBCOLLECTION  = "PriceRecords_full"
PRED_SUBCOLLECTION   = "market_predictions_daily"
WINDOW_LEN           = 20

# --- Firestore init ---
if not firebase_admin._apps:
    cred = credentials.Certificate(SERVICE_ACCOUNT_JSON)
    firebase_admin.initialize_app(cred)
db = firestore.client()

# --- Load Keras RNN (.h5) ---
model = load_model(str(MODEL_PATH), compile=False)
print("✅ Loaded Keras H5 model:", MODEL_PATH)
print("Model input shape:", model.input_shape)
print("Model output shape:", model.output_shape)

✅ Loaded Keras H5 model: out\rnn_model_with_norm.h5
Model input shape: (None, 20, 1)
Model output shape: (None, 1)


In [3]:
# ===================== Cell 2 — Helpers =====================
from datetime import datetime, timezone
import numpy as np

def to_float(x):
    try:
        if isinstance(x, str):
            x = x.replace(",", "")
        return float(x)
    except:
        return np.nan

def parse_date(val):
    if hasattr(val, "to_datetime"):  # Firestore Timestamp
        return val.to_datetime()
    if isinstance(val, datetime):
        return val
    if isinstance(val, str):
        try:
            return datetime.fromisoformat(val)
        except Exception:
            return datetime.strptime(val, "%Y-%m-%d")
    return datetime.now(timezone.utc)

def build_sequences(series_vals, window=WINDOW_LEN):
    X, y = [], []
    for t in range(window, len(series_vals)):
        feat = series_vals[t-window:t]
        target = series_vals[t]
        if np.isnan(feat).any() or np.isnan(target):
            continue
        X.append(feat)
        y.append(target)
    if not X:
        return np.empty((0, window, 1)), np.empty((0,))
    X = np.array(X, dtype="float32")[:, :, None]
    y = np.array(y, dtype="float32")
    return X, y

def predict_next_close(window_vals):
    """Normalize, run model.predict(), denormalize prediction."""
    arr = np.array(window_vals, dtype="float32")
    mu, sigma = np.mean(arr), np.std(arr)
    if not np.isfinite(mu) or sigma <= 1e-12:
        return np.nan

    # normalize window
    x = ((arr - mu) / sigma).astype("float32").reshape(1, WINDOW_LEN, 1)

    # use Keras model
    pred_norm = model.predict(x, verbose=0).reshape(-1)[0]

    # denormalize back
    return float(pred_norm * sigma + mu)

In [4]:
# ===================== Cell 3 — Predict all companies =====================
companies = list(db.collection(COMPANIES_COLLECTION).stream())
print("Companies found:", len(companies))

rows = []
for comp in companies:
    comp_id = comp.id
    price_ref = db.collection(COMPANIES_COLLECTION).document(comp_id).collection(PRICE_SUBCOLLECTION)
    recs = list(price_ref.stream())

    if not recs:
        continue

    # extract closes ordered by date
    tmp = []
    for r in recs:
        d = r.to_dict() or {}
        dt_ = parse_date(d.get("date"))
        close = to_float(d.get("close"))
        tmp.append((dt_, close))
    tmp.sort(key=lambda x: x[0])
    dates, closes = zip(*tmp)

    closes = np.array(closes, dtype="float32")

    # build sliding windows
    X_all, y_all = build_sequences(closes, WINDOW_LEN)

    if len(X_all) == 0:
        continue

    # run predictions for each window
    preds = []
    for i in range(len(X_all)):
        pred_val = predict_next_close(closes[i:i+WINDOW_LEN])
        preds.append(pred_val)

    # align with target dates
    pred_dates = dates[WINDOW_LEN:]  # prediction aligns to t+1 day
    for d, yest, pred in zip(pred_dates, closes[WINDOW_LEN-1:-1], preds):
        recommendation = "Buy" if pred < yest else "Don’t Buy"
        rows.append({
            "company_id": comp_id,
            "date": d.date().isoformat(),
            "yesterday": float(yest),
            "predicted": float(pred),
            "recommendation": recommendation
        })

df = pd.DataFrame(rows)
print("Predictions generated:", len(df))
df.head()

Companies found: 20
Predictions generated: 24560


,company_id,date,yesterday,predicted,recommendation
0,الشركة السعودية للخدمات الارضية,2019-01-29,30.950001,30.871824,Buy
1,الشركة السعودية للخدمات الارضية,2019-01-30,30.799999,30.783920,Buy
2,الشركة السعودية للخدمات الارضية,2019-01-31,30.850000,30.836943,Buy
3,الشركة السعودية للخدمات الارضية,2019-02-03,30.900000,30.891821,Buy
4,الشركة السعودية للخدمات الارضية,2019-02-04,31.250000,31.111467,Buy


In [5]:
# ===================== Cell 4 — Write to Firestore =====================
now_iso = dt.datetime.utcnow().isoformat() + "Z"
BATCH_LIMIT = 500
written = 0

for comp_id, g in df.groupby("company_id"):
    batch = db.batch()
    ops = 0

    company_doc = db.collection(COMPANIES_COLLECTION).document(comp_id)
    pred_col = company_doc.collection(PRED_SUBCOLLECTION)

    for _, r in g.iterrows():
        doc_ref = pred_col.document(r["date"])   # YYYY-MM-DD
        payload = {
            "yesterday": r["yesterday"],
            "predicted": r["predicted"],
            "recommendation": r["recommendation"],
            "updatedAt": now_iso,
        }
        batch.set(doc_ref, payload, merge=True)
        ops += 1
        written += 1

        if ops >= BATCH_LIMIT:
            batch.commit()
            batch = db.batch()
            ops = 0

    if ops:
        batch.commit()

print(f"✅ Wrote {written} docs into subcollections '{PRED_SUBCOLLECTION}' under each company.")

C:\Users\asma5\AppData\Local\Temp\ipykernel_17068\3160227969.py:2: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  now_iso = dt.datetime.utcnow().isoformat() + "Z"


✅ Wrote 24560 docs into subcollections 'market_predictions_daily' under each company.


In [6]:
# ===================== Cell 5 — Diagnostics =====================
print("Total predictions:", len(df))
print("Unique companies:", df["company_id"].nunique())
print("Sample rows:")
print(df.head(10))

Total predictions: 24560
Unique companies: 20
Sample rows:
                        company_id        date  yesterday  predicted  \
0  الشركة السعودية للخدمات الارضية  2019-01-29  30.950001  30.871824   
1  الشركة السعودية للخدمات الارضية  2019-01-30  30.799999  30.783920   
2  الشركة السعودية للخدمات الارضية  2019-01-31  30.850000  30.836943   
3  الشركة السعودية للخدمات الارضية  2019-02-03  30.900000  30.891821   
4  الشركة السعودية للخدمات الارضية  2019-02-04  31.250000  31.111467   
5  الشركة السعودية للخدمات الارضية  2019-02-05  31.250000  31.277872   
6  الشركة السعودية للخدمات الارضية  2019-02-06  31.250000  31.213934   
7  الشركة السعودية للخدمات الارضية  2019-02-07  31.200001  31.162630   
8  الشركة السعودية للخدمات الارضية  2019-02-10  31.049999  31.012804   
9  الشركة السعودية للخدمات الارضية  2019-02-11  31.049999  31.033693   

  recommendation  
0            Buy  
1            Buy  
2            Buy  
3            Buy  
4            Buy  
5      Don’t Buy  
6            Bu

In [ ]:
# ===================== Cell 3 — Predict all companies (no missing dates, PAD=inherit, Arabic recs) =====================
PAD_STRATEGY = "inherit"   # "inherit" or "na"

companies = list(db.collection(COMPANIES_COLLECTION).stream())
print("Companies found:", len(companies))

rows = []
for comp in companies:
    comp_id = comp.id
    price_ref = (
        db.collection(COMPANIES_COLLECTION)
          .document(comp_id)
          .collection(PRICE_SUBCOLLECTION)
    )
    recs = list(price_ref.stream())
    if not recs:
        continue

    # extract closes ordered by date
    tmp = []
    for r in recs:
        d = r.to_dict() or {}
        dt_ = parse_date(d.get("date"))
        close = to_float(d.get("close"))
        tmp.append((dt_, close))
    tmp.sort(key=lambda x: x[0])

    dates, closes = zip(*tmp)  # tuples
    closes = np.array(closes, dtype="float32")
    N = len(closes)

    # Predict for all windows (N - WINDOW_LEN)
    preds = [np.nan] * max(0, N - WINDOW_LEN)
    first_pred = np.nan

    if N >= WINDOW_LEN:
        window_preds = []
        for i in range(N - WINDOW_LEN):
            pv = predict_next_close(closes[i:i+WINDOW_LEN])
            window_preds.append(pv)
        preds = window_preds
        if len(window_preds) > 0 and np.isfinite(window_preds[0]):
            first_pred = float(window_preds[0])

    # Emit exactly one row PER DATE (length N)
    for idx in range(N):
        # normalize date to YYYY-MM-DD
        if hasattr(dates[idx], "date"):
            date_iso = dates[idx].date().isoformat()
        else:
            date_iso = pd.to_datetime(dates[idx]).date().isoformat()

        # yesterday for idx>0
        yesterday_val = float(closes[idx-1]) if idx > 0 and np.isfinite(closes[idx-1]) else None

        if idx < WINDOW_LEN:
            # padding zone (no window yet)
            if PAD_STRATEGY == "inherit" and np.isfinite(first_pred):
                predicted_val = first_pred
                if yesterday_val is None or not np.isfinite(yesterday_val):
                    recommendation = "N/A"
                else:
                    recommendation = "اشتري" if predicted_val < yesterday_val else "لا تشتري"
            else:
                predicted_val = None
                recommendation = "N/A"
        else:
            # normal predicted row aligned so pred(t) uses window ending at t-1
            p = preds[idx - WINDOW_LEN]
            predicted_val = float(p) if np.isfinite(p) else None
            if yesterday_val is None or not np.isfinite(yesterday_val) or predicted_val is None:
                recommendation = "N/A"
            else:
                recommendation = "اشتري" if predicted_val < yesterday_val else "لا تشتري"

        rows.append({
            "company_id": comp_id,
            "date": date_iso,
            "yesterday": yesterday_val if (yesterday_val is None or np.isfinite(yesterday_val)) else None,
            "predicted": predicted_val,
            "recommendation": recommendation
        })

df = pd.DataFrame(rows)
print("Predictions generated (should equal total price rows):", len(df))
df.head(10)

In [6]:
# === Cell 0: init key + project (Firebase Admin) ===
import firebase_admin
from firebase_admin import credentials, firestore as fb_firestore

KEY_PATH = r"nomu-47a92-firebase-adminsdk-fbsvc-1b2e28026c.json"  # ← مسار ملف المفتاح الصحيح
PROJECT_ID = "nomu-47a92"                                         # ← اسم مشروعك الحقيقي

# لو فيه db قديم بمشروع placeholder نزله من الذاكرة
try:
    del db
except NameError:
    pass

# ابدأ Firebase Admin باستخدام ملف المفتاح
if not firebase_admin._apps:
    cred = credentials.Certificate(KEY_PATH)
    firebase_admin.initialize_app(cred, {"projectId": PROJECT_ID})

# أنشئ عميل فايرستور من Firebase Admin (مو من google-cloud مباشرة)
db = fb_firestore.client()

print("✅ Firestore ready for project:", PROJECT_ID)

✅ Firestore ready for project: nomu-47a92


In [7]:
# === Cell 1: quick access test ===
COMPANIES_COLLECTION = "companies"
doc_id = "بنك البلاد"  # تأكد أنه بالضبط نفس الـ document id في قاعدة بياناتك

ref = db.collection(COMPANIES_COLLECTION).document(doc_id)
snap = ref.get()
print("Exists:", snap.exists)
print("Doc path:", ref.path)


Exists: True
Doc path: companies/بنك البلاد


In [8]:
# === Cell 2: delete recommendation for ONE company ===
from firebase_admin import firestore as fb_firestore

PRED_SUBCOLLECTION = "market_predictions_daily"
test_company_id = "بنك البلاد"  # غيّرها حسب ما تريد اختباره

pred_col = db.collection(COMPANIES_COLLECTION).document(test_company_id).collection(PRED_SUBCOLLECTION)
batch = db.batch()
ops = 0
touched = 0

for doc in pred_col.stream():
    batch.update(doc.reference, {"recommendation": fb_firestore.DELETE_FIELD})
    ops += 1
    touched += 1
    if ops >= 500:
        batch.commit()
        batch = db.batch()
        ops = 0

if ops:
    batch.commit()

print(f"✅ تم حذف 'recommendation' من {touched} مستند في {test_company_id}.")

✅ تم حذف 'recommendation' من 1228 مستند في بنك البلاد.


In [9]:
# === Cell 3: full sweep (all companies) ===
from firebase_admin import firestore as fb_firestore

COMPANIES_COLLECTION = "companies"
PRED_SUBCOLLECTION  = "market_predictions_daily"

BATCH_LIMIT = 500
ops = 0
touched = 0
batch = db.batch()

companies = db.collection(COMPANIES_COLLECTION).stream()

for company in companies:
    pred_col = db.collection(COMPANIES_COLLECTION).document(company.id).collection(PRED_SUBCOLLECTION)
    for doc in pred_col.stream():
        batch.update(doc.reference, {"recommendation": fb_firestore.DELETE_FIELD})
        ops += 1
        touched += 1

        if ops >= BATCH_LIMIT:
            batch.commit()
            batch = db.batch()
            ops = 0

if ops:
    batch.commit()

print(f"✅ تم حذف حقل 'recommendation' من {touched} مستند عبر جميع الشركات.")

✅ تم حذف حقل 'recommendation' من 24560 مستند عبر جميع الشركات.
